In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np


In [46]:

# Define the directories
mask_dir = 'data/with_mask'
without_mask_dir = 'data/without_mask'

# Initialize lists to store the images and labels
images = []
labels = []

# Process images with masks
for filename in os.listdir(mask_dir):
    img = Image.open(os.path.join(mask_dir, filename))
    img = img.resize((224, 224))
    images.append(np.array(img))
    labels.append(1)  # Assign label '1' for images with masks

# Process images without masks
for filename in os.listdir(without_mask_dir):
    img = Image.open(os.path.join(without_mask_dir, filename))
    img = img.resize((224, 224))
    images.append(np.array(img))
    labels.append(0)  # Assign label '0' for images without masks

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (7553, 224, 224) + inhomogeneous part.

In [39]:
# Check the shape of each image
for i, img in enumerate(images):
    if img.shape != (224, 224, 3):
        print(f"Image index {i} shape: {img.shape}")


In [44]:
images.shape

(0,)

In [41]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Now, X_train and y_train are your training data and labels
# X_test and y_test are your testing data and labels


In [42]:
X_train.shape

(21, 224, 224, 3)

In [16]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

# Download the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Make the model non-trainable
for layer in base_model.layers:
    layer.trainable = False

# Add a new top layer
x = base_model.output
x = Flatten()(x)  # Flatten dimensions to for use in FC layers
x = Dense(512, activation='relu')(x)  # New FC layer
predictions = Dense(1, activation='sigmoid')(x)  # New softmax layer

# Create a new model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


In [4]:
from tensorflow import keras


In [6]:
#from tensorflow.keras.models import load_model

# Load the model from the file
model = keras.models.load_model('mask_model2.h5')


In [10]:
import cv2
import numpy as np

# To capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    # Read the frame
    _, img = cap.read()

    # Resize and normalize the image
    img_resized = cv2.resize(img, (224, 224))
    img_normalized = np.expand_dims(img_resized, axis=0)

    # Predict the mask
    # ... existing code ...
    
    # Predict the mask
    pred = model.predict(img_normalized)[0][0]
    print(pred)
    
    # Set the threshold
    threshold = 0.65
    
    # Draw the rectangle and text based on the prediction
    if pred < threshold:
        cv2.rectangle(img, (0, 0), (img.shape[1], img.shape[0]), (0, 0, 255), 2)
        cv2.putText(img, 'No Mask', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    else:
        cv2.rectangle(img, (0, 0), (img.shape[1], img.shape[0]), (0, 255, 0), 2)
        cv2.putText(img, 'Mask', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
    
    # ... existing code ...

    # Display
    cv2.imshow('img', img)

    # Stop if escape key is pressed
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break

# Release the VideoCapture object
cap.release()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
0.39807042
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step
0.38995722
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step
0.38851202
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 706ms/step
0.40095872
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 504ms/step
0.40318874
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
0.39516282
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step
0.38637224
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
0.3780439
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step
0.38572034
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
0.37030992
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
0.38303813
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step
0.379758
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
0.40044513
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
0.38927683
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
0.40062308
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
0.39960662
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
0.41083115
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
0.41663462
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
0.46698892
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
0.40868402
1/1

In [28]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


ValueError: Expected all entries in the `metrics` list to be metric objects. Received instead:
metrics=[[<keras.src.metrics.reduction_metrics.MeanMetricWrapper object at 0x0000018E5E145850>]]

In [29]:
import numpy as np

# Choose a test image (replace '0' with the index of the desired test image)
test_image = X_test[0]

# Because the model expects batches, we need to add an extra dimension
test_image = np.expand_dims(test_image, axis=0)

# Make a prediction on the test image
prediction = model.predict(test_image)

# Print out the prediction
print("Prediction:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
Prediction: [[1.]]


In [34]:
import matplotlib.pyplot as plt

# Choose a test image (replace '0' with the index of the desired test image)
test_image = X_test[6]

# Display the image
plt.imshow(test_image)
plt.show()


IndexError: index 6 is out of bounds for axis 0 with size 6

In [36]:
X_test.shape

(6, 224, 224, 3)